# UK Biobank Research Analysis Platform - Basic data extraction
This notebook is mainly based two notebooks from DNAnexus:
https://github.com/dnanexus/OpenBio/blob/master/UKB_notebooks/ukb-rap-pheno-basic.ipynb
https://github.com/dnanexus/UKB_RAP/blob/main/proteomics/0_extract_phenotype_protein_data.ipynb

### This notebook consists of three sections:
### Step 1: Extract clinical phenotype data from dataset['participant'] 
### Step 2: Extract Olink proteomics from dataset['olink_instance_0']
### Step 3: Extract operation records from dataset['hesin_oper']

In [1]:
# Import packages
# dxpy allows python to interact with the platform storage
# Note: This notebook is using spark since the size of the dataset we're extracting
# (i.e. the number of fields) is too large for a single node instance.
import dxpy
import dxdata

import pandas as pd
import subprocess
import glob
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext


# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
# Need to adjust this buffer otherwise will get an error in toPandas() call
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "1024")

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

/cluster/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# Normal spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
# sc = pyspark.SparkContext()
# spark = pyspark.sql.SparkSession(sc)

In [3]:
dxdata.__version__

'0.41.0'

In [4]:
# silence warning
import warnings
warnings.filterwarnings('ignore')

# Re-enable warnings after your code if you want to see warnings again in subsequent cells
# warnings.filterwarnings('default')

In [5]:
# Set the option to None to display all rows, or set a specific number
# pd.set_option('display.max_rows', None)

# Reset display options to default
# pd.reset_option('display.max_rows')

In [6]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

## Access dataset

In [7]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

### Dataset 'entities' are virtual tables linked to one another.

The main entity is 'participant' and corresponds to most pheno fields. Additional entities correspond to linked health care data.
Entities starting with 'hesin' are for hospital records; entities starting with 'gp' are for GP records, etc.

In [8]:
dataset.entities

[<Entity "participant">,
 <Entity "covid19_result_england">,
 <Entity "covid19_result_scotland">,
 <Entity "covid19_result_wales">,
 <Entity "gp_clinical">,
 <Entity "gp_scripts">,
 <Entity "gp_registrations">,
 <Entity "hesin">,
 <Entity "hesin_diag">,
 <Entity "hesin_oper">,
 <Entity "hesin_critical">,
 <Entity "hesin_maternity">,
 <Entity "hesin_delivery">,
 <Entity "hesin_psych">,
 <Entity "death">,
 <Entity "death_cause">,
 <Entity "omop_death">,
 <Entity "omop_device_exposure">,
 <Entity "omop_note">,
 <Entity "omop_observation">,
 <Entity "omop_drug_exposure">,
 <Entity "omop_observation_period">,
 <Entity "omop_person">,
 <Entity "omop_procedure_occurrence">,
 <Entity "omop_specimen">,
 <Entity "omop_visit_detail">,
 <Entity "omop_visit_occurrence">,
 <Entity "omop_dose_era">,
 <Entity "omop_drug_era">,
 <Entity "omop_condition_era">,
 <Entity "omop_condition_occurrence">,
 <Entity "omop_measurement">,
 <Entity "olink_instance_0">,
 <Entity "olink_instance_2">,
 <Entity "olink_

### Step 1: Accessing the main 'participant' entity
The extraction code follows some examples

In [9]:
participant = dataset['participant']

#### Constructing field names, given UKB showcase field id, instance id, and array id

For the main participant data, the Platform uses field names with the following convention:

|Type of field|Syntax for field name|Example|
|:------------|---------------------|-------|
|Neither instanced nor arrayed|`p<FIELD-ID>`|`p31`|
|Instanced but not arrayed|`p<FIELD-ID>_i<INSTANCE-ID>`|`p40005_i0`|
|Arrayed but not instanced|`p<FIELD-ID>_a<ARRAY-ID>`|`p41262_a0`|
|Instanced and arrayed|`p<FIELD-ID>_i<INSTANCE-ID>_a<ARRAY-ID>`|`p93_i0_a0`|

Lastly, the participant id field itself (EID) is named `eid`

If you know exactly the field names you want to work with, put them in a string array (we will see later how to use that):

In [10]:
# example
# field_names = ['eid', 'p31', 'p21022', 'p40005_i0', 'p93_i0_a0']

#### Looking up fields, given UKB showcase field id

If you know the field id but you are not sure if it is instanced or arrayed, and want to grab all instances/arrays (if any), use these:

In [11]:
# Returns all field objects for a given UKB showcase field id

def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UKB showcase field id

def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

##### Examples:

In [12]:
# Participant sex
field_names_for_id('31')

['p31']

In [13]:
# Age when attending assessment centre has multiple instances (visits) 
field_names_for_id('21003')

['p21003_i0', 'p21003_i1', 'p21003_i2', 'p21003_i3']

In [14]:
field_names_for_id('21001') # BMI

['p21001_i0', 'p21001_i1', 'p21001_i2', 'p21001_i3']

In [15]:
field_names_for_id('6150') # having had a heart attack diagnosed by a doctor 

['p6150_i0', 'p6150_i1', 'p6150_i2']

In [16]:
field_names_for_id('20002') # non-cancer illnesses that self-reported as heart attack

['p20002_i0_a0',
 'p20002_i0_a1',
 'p20002_i0_a2',
 'p20002_i0_a3',
 'p20002_i0_a4',
 'p20002_i0_a5',
 'p20002_i0_a6',
 'p20002_i0_a7',
 'p20002_i0_a8',
 'p20002_i0_a9',
 'p20002_i0_a10',
 'p20002_i0_a11',
 'p20002_i0_a12',
 'p20002_i0_a13',
 'p20002_i0_a14',
 'p20002_i0_a15',
 'p20002_i0_a16',
 'p20002_i0_a17',
 'p20002_i0_a18',
 'p20002_i0_a19',
 'p20002_i0_a20',
 'p20002_i0_a21',
 'p20002_i0_a22',
 'p20002_i0_a23',
 'p20002_i0_a24',
 'p20002_i0_a25',
 'p20002_i0_a26',
 'p20002_i0_a27',
 'p20002_i0_a28',
 'p20002_i0_a29',
 'p20002_i0_a30',
 'p20002_i0_a31',
 'p20002_i0_a32',
 'p20002_i0_a33',
 'p20002_i1_a0',
 'p20002_i1_a1',
 'p20002_i1_a2',
 'p20002_i1_a3',
 'p20002_i1_a4',
 'p20002_i1_a5',
 'p20002_i1_a6',
 'p20002_i1_a7',
 'p20002_i1_a8',
 'p20002_i1_a9',
 'p20002_i1_a10',
 'p20002_i1_a11',
 'p20002_i1_a12',
 'p20002_i1_a13',
 'p20002_i1_a14',
 'p20002_i1_a15',
 'p20002_i1_a16',
 'p20002_i1_a17',
 'p20002_i1_a18',
 'p20002_i1_a19',
 'p20002_i1_a20',
 'p20002_i1_a21',
 'p20002_i1_

In [17]:
field_names_for_id('20107') # illness of father

['p20107_i0', 'p20107_i1', 'p20107_i2', 'p20107_i3']

In [18]:
field_names_for_id('20003') # medication

['p20003_i0_a0',
 'p20003_i0_a1',
 'p20003_i0_a2',
 'p20003_i0_a3',
 'p20003_i0_a4',
 'p20003_i0_a5',
 'p20003_i0_a6',
 'p20003_i0_a7',
 'p20003_i0_a8',
 'p20003_i0_a9',
 'p20003_i0_a10',
 'p20003_i0_a11',
 'p20003_i0_a12',
 'p20003_i0_a13',
 'p20003_i0_a14',
 'p20003_i0_a15',
 'p20003_i0_a16',
 'p20003_i0_a17',
 'p20003_i0_a18',
 'p20003_i0_a19',
 'p20003_i0_a20',
 'p20003_i0_a21',
 'p20003_i0_a22',
 'p20003_i0_a23',
 'p20003_i0_a24',
 'p20003_i0_a25',
 'p20003_i0_a26',
 'p20003_i0_a27',
 'p20003_i0_a28',
 'p20003_i0_a29',
 'p20003_i0_a30',
 'p20003_i0_a31',
 'p20003_i0_a32',
 'p20003_i0_a33',
 'p20003_i0_a34',
 'p20003_i0_a35',
 'p20003_i0_a36',
 'p20003_i0_a37',
 'p20003_i0_a38',
 'p20003_i0_a39',
 'p20003_i0_a40',
 'p20003_i0_a41',
 'p20003_i0_a42',
 'p20003_i0_a43',
 'p20003_i0_a44',
 'p20003_i0_a45',
 'p20003_i0_a46',
 'p20003_i0_a47',
 'p20003_i1_a0',
 'p20003_i1_a1',
 'p20003_i1_a2',
 'p20003_i1_a3',
 'p20003_i1_a4',
 'p20003_i1_a5',
 'p20003_i1_a6',
 'p20003_i1_a7',
 'p20003_i

In [19]:
field_names_for_id('191') # date lost to follow up

['p191']

Obtain field name for all instances and arrays for each field_id

### Risk factors
"We defined risk factors at the first assessment as follows: diabetes diagnosed by a doctor (field #2443), BMI (field #21001), current smoking (field #20116), hypertension, family history of heart disease, and high cholesterol. For hypertension we used an expanded definition including self-reported high blood pressure (either on blood pressure medication, data fields #6177, #6153; systolic blood pressure >140 mm Hg, fields #4080, #93; or diastolic blood pressure >90 mm Hg, data fields #4079, #94). For family history of heart disease, we considered history in any first- degree relative (father, mother, sibling; fields #20107, 20110, and 20111, respectively). For high cholesterol, we considered individuals with self- reported high cholesterol at assessment, as well as diagnoses in the HES/death records (ICD-9 272.0; ICD-10 E78.0). For the analyses of the number of elevated risk factors, we considered diagnosed dia- betes (yes/no), hypertension at assessment (yes/no), BMI >30 kg/m2, smoking at assessment (yes/no), high cholesterol (yes/no), and family history of heart disease (yes/no)."

 Am Coll Cardiol. 2018 Oct 16;72(16):1883-1893. 
https://pubmed.ncbi.nlm.nih.gov/30309464/

In [20]:
# risk factor + family history + plate id
field_ids = ['2443', '20116', '6177', '6153', '4080', '93', '4079', '94', 
             '20107', '20110', '20111',
             '30901']
# sum flattens list of lists
sum([field_names_for_id(field_id) for field_id in field_ids], []) 

['p2443_i0',
 'p2443_i1',
 'p2443_i2',
 'p2443_i3',
 'p20116_i0',
 'p20116_i1',
 'p20116_i2',
 'p20116_i3',
 'p6177_i0',
 'p6177_i1',
 'p6177_i2',
 'p6177_i3',
 'p6153_i0',
 'p6153_i1',
 'p6153_i2',
 'p6153_i3',
 'p4080_i0_a0',
 'p4080_i0_a1',
 'p4080_i1_a0',
 'p4080_i1_a1',
 'p4080_i2_a0',
 'p4080_i2_a1',
 'p4080_i3_a0',
 'p4080_i3_a1',
 'p93_i0_a0',
 'p93_i0_a1',
 'p93_i1_a0',
 'p93_i1_a1',
 'p93_i2_a0',
 'p93_i2_a1',
 'p93_i3_a0',
 'p93_i3_a1',
 'p4079_i0_a0',
 'p4079_i0_a1',
 'p4079_i1_a0',
 'p4079_i1_a1',
 'p4079_i2_a0',
 'p4079_i2_a1',
 'p4079_i3_a0',
 'p4079_i3_a1',
 'p94_i0_a0',
 'p94_i0_a1',
 'p94_i1_a0',
 'p94_i1_a1',
 'p94_i2_a0',
 'p94_i2_a1',
 'p94_i3_a0',
 'p94_i3_a1',
 'p20107_i0',
 'p20107_i1',
 'p20107_i2',
 'p20107_i3',
 'p20110_i0',
 'p20110_i1',
 'p20110_i2',
 'p20110_i3',
 'p20111_i0',
 'p20111_i1',
 'p20111_i2',
 'p20111_i3',
 'p30901_i0',
 'p30901_i1',
 'p30901_i2',
 'p30901_i3']

#### Looking up fields by title keyword

If you remember part of the field title, use these:

In [21]:
# Returns all field objects for a given title keyword

def fields_by_title_keyword(keyword):
    from distutils.version import LooseVersion
    fields = list(participant.find_fields(lambda f: keyword.lower() in f.title.lower()))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given title keyword

def field_names_by_title_keyword(keyword):
    return [f.name for f in fields_by_title_keyword(keyword)]

# Returns all field titles for a given title keyword

def field_titles_by_title_keyword(keyword):
    return [f.title for f in fields_by_title_keyword(keyword)]

# Furhter information: https://github.com/dnanexus/OpenBio/blob/master/UKB_notebooks/ukb-rap-pheno-basic.ipynb

### Grabbing fields into a Spark DataFrame

The `participant.retrieve_fields` function can be used to construct a Spark DataFrame of the given fields.

By default, this retrieves data as encoded by UK Biobank. For example, field p31 (participant sex) will be returned as an integer column with values of 0 and 1. To receive decoded values, supply the `coding_values='replace'` argument.

For more information, see [Tips for Retrieving Fields](https://dnanexus.gitbook.io/uk-biobank-rap/getting-started/working-with-ukb-data#tips-for-retrieving-fields) in the documentation.

## Extract clinical data 

In [22]:
field_names = ['eid', 
               'p31',  # sex
               'p21022',  # age at recruitment
               'p21001_i0',  # BMI
               'p54_i0',  # UK Biobank assessment centre
               'p53_i0',  # Date of attending assessment centre p53_i0
               'p40000_i0',  # Date of death
               'p191', # date lost to follow-up
               'p20003_i0_a0', # medication
               'p6150_i0',  # having had a heart attack diagnosed by a doctor 
               'p20002_i0_a0',  # non-cancer illnesses that self-reported as heart attack 
               'p20004_i0_a0',  # self-reported operation including PTCA, CABG, or triple heart bypass
               'p2443_i0',  # diabetes
               'p20116_i0',  # current smoking
               'p20160_i0', # ever smoked
               'p6177_i0',  # self-reported high blood pressure
               'p6153_i0',  # self-reported high blood pressure
               'p4080_i0_a0',  # systolic blood pressure 
               'p93_i0_a0',  # systolic blood pressure 
               'p4079_i0_a0',  # diastolic blood pressure
               'p94_i0_a0', # diastolic blood pressure
               'p6177_i0', # Medication for cholesterol, blood pressure or diabetes | Instance 0
               'p20107_i0', #illness of father
               'p20110_i1', # illness of mother
               'p20111_i0', # illness of sibling
               'p30901_i0', # olink plateID
              ] \
                + field_names_for_id('41270') \
                + field_names_for_id('41280')  # Corrected line

# 41270 = ICD10

• Further informatiaon on Date of first in-patient diagnosis can be found at https://biobank.ndph.ox.ac.uk/crystal/field.cgi?id=41280:
The corresponding ICD-10 diagnosis codes can be found in data-field Field 41270 and the two fields can be linked using the array structure.

• CAD definition and risk factors:
J Am Coll Cardiol. 2018 Oct 16;72(16):1883-1893. 
https://pubmed.ncbi.nlm.nih.gov/30309464/


## Grabbing fields into a Spark DataFrame
The participant.retrieve_fields function can be used to construct a Spark DataFrame of the given fields.

By default, this retrieves data as encoded by UK Biobank. For example, field p31 (participant sex) will be returned as an integer column with values of 0 and 1. To receive decoded values, supply the coding_values='replace' argument.

For more information, see Tips for Retrieving Fields in the documentation.

In [23]:
# Grabbing fields into a Spark DataFrame
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

In [24]:
# See the first five entries as a Spark DataFrame:
# df.show(5, truncate=False)

In [25]:
# See the first five entries as a Pandas DataFrame:
df.limit(5).toPandas()

,eid,p31,p21022,p21001_i0,p54_i0,p53_i0,p40000_i0,p191,p20003_i0_a0,p6150_i0,...,p41280_a249,p41280_a250,p41280_a251,p41280_a252,p41280_a253,p41280_a254,p41280_a255,p41280_a256,p41280_a257,p41280_a258
0,2789877,1,56,22.6630,11004,2008-04-07,None,None,1.140868e+09,[-7],...,None,None,None,None,None,None,None,None,None,None
1,3070454,1,61,25.3439,11009,2008-03-14,2012-06-03,None,NaN,[-7],...,None,None,None,None,None,None,None,None,None,None
2,2733030,0,59,24.8491,11016,2009-04-08,None,None,NaN,[-7],...,None,None,None,None,None,None,None,None,None,None
3,3039657,1,57,29.3577,10003,2006-03-21,None,None,1.140880e+09,[4],...,None,None,None,None,None,None,None,None,None,None
4,4404117,0,49,34.1036,11014,2010-06-22,None,None,1.140917e+09,[-7],...,None,None,None,None,None,None,None,None,None,None


In [26]:
# if the above looks good, go ahead and convert the entire spark data frame to pandas data frame 
pdf = df.toPandas()

In [27]:
print(pdf.columns)

Index(['eid', 'p31', 'p21022', 'p21001_i0', 'p54_i0', 'p53_i0', 'p40000_i0',
       'p191', 'p20003_i0_a0', 'p6150_i0',
       ...
       'p41280_a249', 'p41280_a250', 'p41280_a251', 'p41280_a252',
       'p41280_a253', 'p41280_a254', 'p41280_a255', 'p41280_a256',
       'p41280_a257', 'p41280_a258'],
      dtype='object', length=285)


In [28]:
# Set the option to None to display all rows, or set a specific number
# pd.set_option('display.max_rows', None)

# Your code to display the Series or DataFrame
# For example, if you have a Series named non_na_counts:
# pdf.count()

In [29]:
## Given the information above, re-select the column if needed
#field_names = ['p31', 'p21022', 'p54_i0', 'p41202', 'p53_i0', 'p40000_i0']  \
#    + field_names_for_id('41280') 
#df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

In [30]:
# Reset display options to default
# pd.reset_option('display.max_rows')

In [31]:
# Saving as TSV file
pdf.to_csv('clinical_data.tsv', sep='\t', index=False)

### Step 2: Extract Olink proteomics from dataset['olink_instance_0']

In [32]:
olink = dataset['olink_instance_0']
# olink.fields # to list all

In [33]:
temp_list = olink.fields
type(temp_list)

list

In [34]:
#def field_names_for_id(field_id):
#    return [f.name for f in fields_for_id(field_id)]
olink_all_field_names = [f.name for f in olink.fields]

In [37]:
#field_names = ['eid', 'col6a3'] # select like this if you're interested in only a few protein
olink_all_field_names[:5] # check

['eid', 'a1bg', 'aamdc', 'aarsd1', 'abca2']

In [38]:
dfo = olink.retrieve_fields(names=olink_all_field_names, engine=dxdata.connect())

In [ ]:
# to check
# dfo.head(5)

### In case of extracting particular proteins only:
olink_field_names = ['eid', 'col6a3']
dfo = olink.retrieve_fields(names=olink_field_names, engine=dxdata.connect())

In [ ]:
# See the first five entries as a Pandas DataFrame:
# dfo.limit(5).toPandas()

In [44]:
dfo.count() # check rows (53016 individual's data)

53016

In [45]:
pdfo = dfo.toPandas()

In [47]:
pdfo.shape # (53016, 2924)

(53016, 2924)

In [43]:
# Alternative approach
# pdfo = dfo.toPandas() is very memory intensive. So instead, we can do this sequentially (if needed).
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window

# Total number of rows in the DataFrame
total_rows = dfo.count()

# Number of chunks
num_chunks = 10

# Calculate the number of rows per chunk. Adding 1 to ensure the last chunk includes all remaining rows
rows_per_chunk = (total_rows // num_chunks) + (total_rows % num_chunks > 0)

# Initialize an empty list to store each chunk's pandas DataFrame
chunks_list = []

# Create a column 'row_id' to help in filtering rows for each chunk
dfo_with_id = dfo.withColumn("row_id", row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

for i in range(num_chunks):
    # Calculate start index for the current chunk
    start_index = i * rows_per_chunk
    
    # End index is not needed as we limit the number of rows fetched
    chunk_df = dfo_with_id.filter(dfo_with_id.row_id >= start_index).limit(rows_per_chunk)
    
    # Convert the chunk to a pandas DataFrame and append to the list
    chunk_pd_df = chunk_df.drop("row_id").toPandas()
    chunks_list.append(chunk_pd_df)

# Concatenate all chunks to form the final pandas DataFrame
pdf = pd.concat(chunks_list, ignore_index=True)

# Checking the shape of the final DataFrame
print(pdf.shape)

[root] ERROR: KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/cluster/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/cluster/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
pdfo.iloc[:5, :5] # check

In [ ]:
pdfo.to_csv('olink_data.tsv', sep='\t', index=False)

### Step 3: Extract operation records from dataset['hesin_oper']

In [48]:
operation = dataset['hesin_oper']

In [49]:
# operation.fields[1:5] # check
operation.fields

[<Field "dnx_hesin_oper_id">,
 <Field "dnx_hesin_id">,
 <Field "eid">,
 <Field "ins_index">,
 <Field "preopdur">,
 <Field "arr_index">,
 <Field "level">,
 <Field "opdate">,
 <Field "oper3">,
 <Field "oper3_nb">,
 <Field "oper4">,
 <Field "oper4_nb">,
 <Field "posopdur">]

In [50]:
operation_all_field_names = [f.name for f in operation.fields]
print(operation_all_field_names)

['dnx_hesin_oper_id', 'dnx_hesin_id', 'eid', 'ins_index', 'preopdur', 'arr_index', 'level', 'opdate', 'oper3', 'oper3_nb', 'oper4', 'oper4_nb', 'posopdur']


In [51]:
df_ope = operation.retrieve_fields(names=operation_all_field_names, engine=dxdata.connect())

In [52]:
df_ope.head(5)

[Row(dnx_hesin_oper_id='4631925-4-0', dnx_hesin_id='4631925-4', eid='4631925', ins_index=4, preopdur=0, arr_index=0, level='1', opdate=datetime.date(2015, 12, 4), oper3=None, oper3_nb=None, oper4='W822', oper4_nb=None, posopdur=0),
 Row(dnx_hesin_oper_id='3187543-12-8', dnx_hesin_id='3187543-12', eid='3187543', ins_index=12, preopdur=None, arr_index=8, level='2', opdate=datetime.date(2016, 2, 26), oper3=None, oper3_nb=None, oper4='K634', oper4_nb=None, posopdur=None),
 Row(dnx_hesin_oper_id='5471173-4-14', dnx_hesin_id='5471173-4', eid='5471173', ins_index=4, preopdur=None, arr_index=14, level='2', opdate=datetime.date(2016, 9, 27), oper3=None, oper3_nb=None, oper4='E492', oper4_nb=None, posopdur=None),
 Row(dnx_hesin_oper_id='2559576-0-2', dnx_hesin_id='2559576-0', eid='2559576', ins_index=0, preopdur=None, arr_index=2, level='2', opdate=datetime.date(2011, 9, 1), oper3=None, oper3_nb=None, oper4='Y819', oper4_nb=None, posopdur=None),
 Row(dnx_hesin_oper_id='1886597-0-4', dnx_hesin_id

In [53]:
df_ope.limit(5).toPandas().head() # check

,dnx_hesin_oper_id,dnx_hesin_id,eid,ins_index,preopdur,arr_index,level,opdate,oper3,oper3_nb,oper4,oper4_nb,posopdur
0,4631925-4-0,4631925-4,4631925,4,0.0,0,1,2015-12-04,None,None,W822,None,0.0
1,3187543-12-8,3187543-12,3187543,12,NaN,8,2,2016-02-26,None,None,K634,None,NaN
2,5471173-4-14,5471173-4,5471173,4,NaN,14,2,2016-09-27,None,None,E492,None,NaN
3,2559576-0-2,2559576-0,2559576,0,NaN,2,2,2011-09-01,None,None,Y819,None,NaN
4,1886597-0-4,1886597-0,1886597,0,NaN,4,2,2014-01-29,None,None,Y767,None,NaN


In [38]:
# this is too memory intensive
# pdf_ope = df_ope.dropna(subset=['oper4']).toPandas()
# pdf_ope.to_csv('operation_data.tsv', sep = '\t', index = False)

In [55]:
# Alternatively, we do this sequentially
# from pyspark.sql.functions import monotonically_increasing_id, row_number
# from pyspark.sql.window import Window

# Total number of rows in the DataFrame
total_rows = df_ope.count()

# Number of chunks
num_chunks = 10

# Calculate the number of rows per chunk. Adding 1 to ensure the last chunk includes all remaining rows
rows_per_chunk = (total_rows // num_chunks) + (total_rows % num_chunks > 0)

# Initialize an empty list to store each chunk's pandas DataFrame
chunks_list = []

# Create a column 'row_id' to help in filtering rows for each chunk
df_ope_with_id = df_ope.withColumn("row_id", row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

for i in range(num_chunks):
    # Calculate start index for the current chunk
    start_index = i * rows_per_chunk
    
    # End index is not needed as we limit the number of rows fetched
    chunk_df = df_ope_with_id.filter(df_ope_with_id.row_id >= start_index).limit(rows_per_chunk)
    
    # Convert the chunk to a pandas DataFrame and append to the list
    chunk_pd_df = chunk_df.drop("row_id").toPandas()
    chunks_list.append(chunk_pd_df)

# Concatenate all chunks to form the final pandas DataFrame
pdf_ope = pd.concat(chunks_list, ignore_index=True)

# Checking the shape of the final DataFrame
print(pdf_ope.shape)

(8763355, 13)


In [57]:
pdf_ope.to_csv('operation_data.tsv', sep = '\t', index = False)

In [39]:
# hesin is optional
# hesin = dataset['hesin']
# hesin_all_field_names = [f.name for f in hesin.fields]
# print(hesin_all_field_names)
# df_hesin = hesin.retrieve_fields(names=hesin_all_field_names, engine=dxdata.connect())
# df_hesin.limit(5).toPandas().head() # check

In [58]:
# retrive hesin_diag, which contains ICD10
hesin_diag = dataset['hesin_diag']
hesin_diag_all_field_names = [f.name for f in hesin_diag.fields]
print(hesin_diag_all_field_names)
df_hesin_diag = hesin_diag.retrieve_fields(names=hesin_diag_all_field_names, engine=dxdata.connect())
df_hesin_diag.limit(5).toPandas().head() # check

['dnx_hesin_diag_id', 'dnx_hesin_id', 'eid', 'ins_index', 'arr_index', 'level', 'diag_icd9', 'diag_icd9_nb', 'diag_icd10', 'diag_icd10_nb']


,dnx_hesin_diag_id,dnx_hesin_id,eid,ins_index,arr_index,level,diag_icd9,diag_icd9_nb,diag_icd10,diag_icd10_nb
0,4816350-0-0,4816350-0,4816350,0,0,1,None,None,D140,None
1,3835670-9-2,3835670-9,3835670,9,2,2,None,None,Z800,None
2,2693369-4-8,2693369-4,2693369,4,8,2,None,None,N183,None
3,3888853-17-11,3888853-17,3888853,17,11,2,None,None,Z888,None
4,5787855-42-1,5787855-42,5787855,42,1,2,None,None,M059,None


In [59]:
# again, toPandas() is too memory intensive
# pdf_hesin_diag = df_hesin_diag.dropna(subset = ['diag_icd10']).toPandas()
# Alternatively, we do this sequentially
# from pyspark.sql.functions import monotonically_increasing_id, row_number
# from pyspark.sql.window import Window

# Total number of rows in the DataFrame
total_rows = df_hesin_diag.count()

# Number of chunks
num_chunks = 10

# Calculate the number of rows per chunk. Adding 1 to ensure the last chunk includes all remaining rows
rows_per_chunk = (total_rows // num_chunks) + (total_rows % num_chunks > 0)

# Initialize an empty list to store each chunk's pandas DataFrame
chunks_list = []

# Create a column 'row_id' to help in filtering rows for each chunk
df_hesin_diag_with_id = df_hesin_diag.withColumn("row_id", row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

for i in range(num_chunks):
    # Calculate start index for the current chunk
    start_index = i * rows_per_chunk
    
    # End index is not needed as we limit the number of rows fetched
    chunk_df = df_hesin_diag_with_id.filter(df_hesin_diag_with_id.row_id >= start_index).limit(rows_per_chunk)
    
    # Convert the chunk to a pandas DataFrame and append to the list
    chunk_pd_df = chunk_df.drop("row_id").toPandas()
    chunks_list.append(chunk_pd_df)

# Concatenate all chunks to form the final pandas DataFrame
pdf_hesin_diag = pd.concat(chunks_list, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'shape'

In [61]:
print(pdf_hesin_diag.shape)

(17494320, 10)


In [62]:
pdf_hesin_diag.to_csv('operation_icd10_data.tsv', sep = '\t', index = False)

In [ ]:
# merge df_ope and df_hesin_diag (this is too memory intensive)
# pdf_ope_diag = pd.merge(pdf_ope, pdf_hesin_diag, on='eid', how='inner')
# pdf_ope_diag.head() # check
# pdf_ope_diag.to_csv('operation_data_with_icd10.tsv', sep = '\t', index = False)

# retrive death record
hesin = dataset['hesin']
hesin_all_field_names = [f.name for f in hesin.fields]
print(hesin_all_field_names)
df_hesin = hesin.retrieve_fields(names=hesin_all_field_names, engine=dxdata.connect())
df_hesin.limit(5).toPandas().head() # check#### Step 4: Extract death record
4.1. <Entity "death">,
4.2 <Entity "death_cause">

In [39]:
death = dataset['death']

In [40]:
death.fields

[<Field "dnx_death_id">,
 <Field "eid">,
 <Field "ins_index">,
 <Field "dsource">,
 <Field "source">,
 <Field "date_of_death">]

In [41]:
death_all_field_names = [f.name for f in death.fields]
print(death_all_field_names)
df_death = death.retrieve_fields(names=death_all_field_names, engine=dxdata.connect())
df_death.head(5)

['dnx_death_id', 'eid', 'ins_index', 'dsource', 'source', 'date_of_death']


[Row(dnx_death_id='4590254-0', eid='4590254', ins_index=0, dsource='E/W', source='1', date_of_death=datetime.date(2009, 11, 25)),
 Row(dnx_death_id='3989605-0', eid='3989605', ins_index=0, dsource='E/W', source='2', date_of_death=datetime.date(2019, 11, 29)),
 Row(dnx_death_id='2042144-0', eid='2042144', ins_index=0, dsource='E/W', source='2', date_of_death=datetime.date(2015, 10, 20)),
 Row(dnx_death_id='5651155-0', eid='5651155', ins_index=0, dsource='SCOT', source='55', date_of_death=datetime.date(2020, 10, 18)),
 Row(dnx_death_id='3253040-0', eid='3253040', ins_index=0, dsource='E/W', source='2', date_of_death=datetime.date(2016, 10, 14))]

In [ ]:
pdf_death = df_death.toPandas() # convert to pandas data frame

In [ ]:
pdf_death.to_csv('death_data.tsv', sep= '\t', index= False)

In [47]:
# 4.2
death_cause = dataset['death_cause']

In [50]:
print(death_cause.fields)

[<Field "dnx_death_cause_id">, <Field "dnx_death_id">, <Field "eid">, <Field "ins_index">, <Field "arr_index">, <Field "level">, <Field "cause_icd10">]


In [51]:
death_cause_all_field_names = [f.name for f in death_cause.fields]
print(death_cause_all_field_names)
df_death_cause = death_cause.retrieve_fields(names=death_cause_all_field_names, engine=dxdata.connect())
df_death_cause.head(5)

['dnx_death_cause_id', 'dnx_death_id', 'eid', 'ins_index', 'arr_index', 'level', 'cause_icd10']


[Row(dnx_death_cause_id='1910421-0-4', dnx_death_id='1910421-0', eid='1910421', ins_index=0, arr_index=4, level=2, cause_icd10='I38'),
 Row(dnx_death_cause_id='1338497-0-1', dnx_death_id='1338497-0', eid='1338497', ins_index=0, arr_index=1, level=2, cause_icd10='F03'),
 Row(dnx_death_cause_id='1451607-0-0', dnx_death_id='1451607-0', eid='1451607', ins_index=0, arr_index=0, level=1, cause_icd10='U071'),
 Row(dnx_death_cause_id='4443217-0-1', dnx_death_id='4443217-0', eid='4443217', ins_index=0, arr_index=1, level=2, cause_icd10='R58'),
 Row(dnx_death_cause_id='1043906-0-0', dnx_death_id='1043906-0', eid='1043906', ins_index=0, arr_index=0, level=1, cause_icd10='I608')]

In [53]:
pdf_death_cause = df_death_cause.toPandas() # convert to pandas data frame

In [54]:
pdf_death_cause.to_csv('death_cause_data.tsv', sep= '\t', index= False)

### upload the resultant files and the current notebook (after saving it)

In [ ]:
%%bash
dx upload clinical_data.tsv --dest UKB:/data/03.incident_CAD/
dx upload olink_data.tsv --dest UKB:/data/03.incident_CAD/
dx upload operation_data.tsv --dest UKB:/data/03.incident_CAD/
dx upload operation_icd10_data.tsv --dest UKB:/data/03.incident_CAD/
dx upload death_data.tsv --dest UKB:/data/03.incident_CAD/
dx upload death_cause_data.tsv --dest UKB:/data/03.incident_CAD/
dx upload ukbrap_extract_clinical_olink_ope_data.ipynb --dest UKB:/data/03.incident_CAD/

ID                          file-Gj68840JBPpg9y5FvKQbKzQ7
Class                       file
Project                     project-Gf7Jk4QJBPpxxxG2x187pyJg
Folder                      /data/03.incident_CAD
Name                        clinical_data.tsv
State                       closing
Visibility                  visible
Types                       -
Properties                  -
Tags                        -
Outgoing links              -
Created                     Tue Apr  2 22:17:52 2024
Created by                  satoshi.yoshiji
 via the job                job-Gj62ZpjJBPpk8zQ88x4V4b0K
Last modified               Tue Apr  2 22:17:54 2024
Media type                  
archivalState               "live"
cloudAccount                "cloudaccount-dnanexus"


## Below are miscellaneous codes

### if you want to combine data frame, use pandas's merge 
combined = pd.merge(pdf, df_final, on = 'eid', how= 'left')

### Filtering spark dataframes

Spark dataframes can be filtered using the syntax: `df.filter(expression)`

The expression can be either :

* a string expression, built using SQL field names (e.g. `p31`) and SQL operators (e.g. `=`, `NOT`, `OR`, `AND`)
  * example: `'(p21022 >= 50) AND (p31 = 0)'`
  

* a Python expression, built using Python object fields (e.g. `df.p31`) and Python operators (e.g. `==`, `!`, `|`, `&`)
  * example: `(df.p21022 >= 50) & (df.p31 == 0)`

#### Example: Participants above 50 years old and female

df.count()

### Using SQL syntax
df.filter('(p21022 >= 50) AND (p31 = 0)').count()

### Using Python syntax
df.filter((df.p21022 >= 50) & (df.p31 == 0)).count()

### Getting more information about fields

Field objects (such as those returned by `fields_for_id` and `fields_by_title_keyword` above, or via the `participant[<field_name>]` syntax) provide additional information such as codings and units.

####  Working with codings

participant['p31'].coding

participant['p31'].coding.codes

def field_codes_by_keyword(field_name, keyword):
    return dict([(k,v) for (k,v) in participant[field_name].coding.codes.items() if keyword.lower() in v.lower()])

field_codes_by_keyword('p31', 'female')

field_codes_by_keyword('p41202', 'obesity')

### Additional field information

### Get link to UKB documentation page
participant['p31'].linkout

### Get field units
participant['p21022'].units

### Saving results

### Saving as CSV file
df.toPandas().to_csv('results.csv', index=False)

### Saving as TSV file
df.toPandas().to_csv('results.tsv', sep='\t', index=False)

### Saving as DTA file (Stata)
df.toPandas().astype(str).replace('None|NaN|nan', '.', regex=True).to_stata('results.dta')

#### Writing results back to the project

%%bash
dx upload results.tsv --dest / 